<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex04notebookC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex04notebookC

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 演習: 勾配法によるパラメータの最適化
----




----
### 準備


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import pandas as pd
import seaborn
seaborn.set()

----
### 問題1

関数 $E(w_1, w_2)$ を
$$
\begin{aligned}
E(w_1, w_2) &= \frac{(2w_2-w_1^2)^2}{4} + \frac{(1-w_1)^2}{8}
\end{aligned}
$$
と定める．このとき，$\frac{\partial E}{\partial w_1}, \frac{\partial E}{\partial w_2}$ を計算し，
$\frac{\partial E}{\partial w_1} = \frac{\partial E}{\partial w_2} = 0$ を満たす $(w_1, w_2)$ を求めなさい．

---

次の2つのセルを実行すると，$E(w_1, w_2)$ の概形を描かせることができます．

In [ ]:
# E(w1, w2)
def E(w):
    return (2*w[1] - w[0]*w[0])**2/4 + (1 - w[0])**2/8

# E(w1, w2) の w1, w2 に関する偏微係数
def dEdw(w):
    return np.array([0, 0]) ### 要修正

In [ ]:
# 2変数の具体例のグラフ
fig = plt.figure(facecolor='white', figsize=(12, 6))

xmin, xmax = -1, 2
ymin, ymax = -1, 2

# (w1, w2) に対する E(w1, w2) の計算
w1, w2 = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))
w1w2 = np.vstack((w1.ravel(), w2.ravel())).T
Ew1w2 = np.array([E(w) for w in w1w2])
EE = Ew1w2.reshape((w1.shape[0], w2.shape[1]))

# 三次元プロット
elevation = 20
azimuth = -70
ax0 = fig.add_subplot(121, projection='3d')
ax0.plot_wireframe(w1, w2, EE)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_zlim(-1, 5)
ax0.set_xlabel('$w_1$')
ax0.set_ylabel('$w_2$')
ax0.set_zlabel('$E(w_1, w_2)$')
ax0.view_init(elevation, azimuth)

# 二次元等高線プロット
ax1 = fig.add_subplot(122)
cval = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]
contour = ax1.contour(w1, w2, EE, cval)
ax1.clabel(contour, fontsize=10)
ax1.set_aspect('equal')
ax1.set_xlabel('$w_1$')
ax1.set_ylabel('$w_2$')
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(xmin, xmax)

plt.tight_layout()
plt.show()

----
### 問題2

関数 $E(w_1, w_2)$ の最小値を最急降下法で求めてみましょう．

関数 `dEdw` を定義しているセルの `### 要修正` と書かれた行を修正してから次のセルを実行すると，$(w_1, w_2) = (0, 1)$ を初期値として $E(w_1, w_2)$ が最小になる $(w_1, w_2)$ を探すことができます．正しく動作することを確認しましょう．

---


In [ ]:
w = np.array([0.0, 1.0]) # w_1, w_2 の初期値
eta = 0.3                # 学習係数

# 最急降下法の繰り返し
for i in range(100):
    print(f'step{i}: w = {w}, E(w) = {E(w):.4f}')
    dw = dEdw(w)  # 勾配の値の計算
    w -= eta*dw  # パラメータの更新
print(f'step{i}: w = {w}, E(w) = {E(w):.4f}')

次のセルたちを実行すると，最急降下法で $(w_1, w_2)$ の値が更新されていく様子をアニメーションで見ることができます．やってみよう．

動作確認できたら，初期値や学習係数の値をいろいろ変えて実験してみよう．

In [ ]:
def genAnim2D(f, dfdx, x0, eta):

    fig, ax = plt.subplots(facecolor="white", figsize=(6, 6))

    # f(x1, x2) の等高線を描く
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))
    XX = np.vstack((xx.ravel(), yy.ravel())).T
    ZZ = np.array([f(x) for x in XX])
    zz = ZZ.reshape((xx.shape[0], xx.shape[1]))
    cval = [0, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 3, 4, 5]
    contour = ax.contour(xx, yy, zz, cval)
    ax.clabel(contour, fontsize=10)
    ax.set_aspect('equal')
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

    # アニメーションの各コマを生成
    aList = []
    x = x0
    for i in range(50):
        a1 = ax.plot(x[0], x[1], marker='o', markersize=12, color='red')
        s = f'step{i}: $E = {f(x):.3f}$'
        a2 = ax.text(-0.9, 1.5, s, size=20)
        a1.append(a2)
        aList.append(a1)
        # 最急降下法
        dx = dfdx(x)
        x -= eta*dx

    anim = animation.ArtistAnimation(fig, aList, interval=300)
    rc('animation', html='jshtml')
    plt.close()

    return anim

In [ ]:
w = np.array([0.0, 1.0]) # w_1, w_2 の初期値
eta = 0.3                # 学習係数
anim = genAnim2D(E, dEdw, w, eta)
anim